# Diffie-Hellman Key Exchange

State of the art ciphers use keys to encrypt and decrypt messages. The key is a number which is agreed upon before the parties exchange their messages. Only knowledge of the chosen key allows to decrypt the encrypted messages. Therefore, it is crucial that the key is kept secret at all moments. This begs the question: **how can two parties agree on a secret key when they must fear that all their communication is intercepted?**

In [359]:
def exponentiationBySquaring(a, b, modulus):
    # b must be a non-negative integer
    decomposition = list()
    exponent = b
    
    while exponent > 0:
        if exponent % 2 == 0:
            decomposition.insert(0, True)
            exponent = exponent // 2
        elif exponent % 2 == 1:
            decomposition.insert(0, False)
            exponent = exponent - 1

    result = 1 % modulus

    for i in decomposition:
        if i:
            result = (result ** 2) % modulus
        else:
            result = (result * a) % modulus

    return result


In [392]:
def encryptKey(privateKey, publicKey1, publicKey2):
    return exponentiationBySquaring(publicKey1, privateKey, publicKey2)

In [393]:
def computeCommonKey(privateKey, encryptedKey, publicKey2):
    return exponentiationBySquaring(encryptedKey, privateKey, publicKey2)

We assume that two parties, Alice and Bob, want to find a common key.

First, Alice and Bob choose two keys. This may happen publicly, since those keys are distinct from the key that is generated in the end.

In [362]:
public1 = 6
public2 = 761

Now, Alice and Bob, choose one private key each.

In [363]:
alicePrivate = 630
bobPrivate = 694

Those keys are to be kept secret at all time: only Alice knows her key, and only Bob knows his key.
So, Alice and Bob encrpyt their respective keys, before they sent it to each other.

In [394]:
aliceEncrypted = encryptKey(alicePrivate, public1, public2)
bobEncrypted = encryptKey(bobPrivate, public1, public2)
print('Alice private key:', alicePrivate, '-> encrypted:', aliceEncrypted)
print('Bob private key:', bobPrivate, '-> encrypted:', bobEncrypted)

Alice private key: 630 -> encrypted: 716
Bob private key: 694 -> encrypted: 144


Finally, the common key is defined to be:

In [365]:
commonKey = exponentiationBySquaring(public1, alicePrivate ** bobPrivate, public2)
print('common key:', commonKey)

common key: 405


But Alice and Bob don't know each other's private Key, only its encrypted version. However, this is enough to compute the common key.

In [366]:
commonKey_Alice = computeCommonKey(alicePrivate, bobEncrypted, public2)
print('Alice computes the common key:', commonKey_Alice)
commonKey_Bob = computeCommonKey(bobPrivate, aliceEncrypted, public2)
print('Bob computes the common key:', commonKey_Bob)

Alice computes the common key: 207
Bob computes the common key: 207


## Attacks

An attacker might try to commpute the common key by decoding either Alice's of Bob's private key. Let's try to find Alice's private key by trying all possiblities.

In [401]:
def decryptKey(encryptedKey, publicKey1, publicKey2):
    for possibleKey in range(0, publicKey2 - 1):
        if encryptKey(possibleKey, publicKey1, publicKey2) == encryptedKey:
            return possibleKey

In [368]:
aliceDecrypted = decryptKey(aliceEncrypted, public1, public2)
print('Alice private key has been hacked:', aliceDecrypted)

Alice private key has been hacked: 630


## Security

In our example, it was easy for an attacker to decrypt Alice's private key by trying every possibility. This is why in practice the public keys and private keys are chosen to be much longer: at least 2048 bits is recommended!

There is one more thing to be considered when choosing the public keys. Let's assume for a moment that we had taken a bad choice for $\mathrm{public1}$.

In [369]:
public1 = 1

This would be a serious security risk, because the common key would be $1$ no matter what Alice's and Bob's private keys are.

In [ ]:
for i in range(0, public2):
    print(exponentiationBySquaring(public1, i, public2))

The public keys in our example are better, because the common key can actually attain any value from $1$ to $\mathrm{public2} - 1$.

In [ ]:
public1 = 6
for i in range(0, public2):
    print(exponentiationBySquaring(public1, i, public2))

Mathematicians say that $6$ is a *primitive root modulo $761$* and a fancy mathematical theorem enssures that such a root of unity exists, since $761$ is a prime number.

### Larger keys are needed!

In [372]:
import math
import random

In [373]:
def isPrime(number):
    # onyl works for integers > 1
    searchspace = math.trunc(math.sqrt(number))
    
    for divisor in range(2, searchspace + 1):
        if number % divisor == 0:
            return False
        
    return True

In [374]:
def randomPrime(upperbound):
    
    randomInt = random.randint(2, upperbound)
    
    while not isPrime(randomInt):
        randomInt = random.randint(2, upperbound) # picks a random number between 2 and upperbound (both included)

    return randomInt

In [385]:
def isPrimitiveRootModulo(number, k, n):
    if exponentiationBySquaring(number, k, n) != 1:
        return False
    else:
        for i in range(2, k // 2 + 1): # try all prime divisors of k
            if k % i == 0:
                if isPrime(i):
                    exponent = exponentiationBySquaring(number, k // i, n)
                    if exponent == 1:
                        return False
    
    return True
                    
                

In [386]:
def randomPrimitiveRootModulo(modulus):

    randomInt = random.randint(1, modulus - 1)

    while not isPrimitiveRootModulo(randomInt, modulus - 1, modulus):
        randomInt = random.randint(1, modulus - 1)
    
    return randomInt

In [404]:
def lowestPrimitiveRootModulo(modulus):
    for i in range(1, modulus - 1):
        if isPrimitiveRootModulo(i, modulus - 1, modulus):
            return i

In [409]:
prime = randomPrime(2 ** 32)
print(prime)

1900476043


In [410]:
primitiveRoot = lowestPrimitiveRootModulo(prime)
print(primitiveRoot)

2


In [411]:
alicePrivate = random.randint(1, prime)
aliceEncrypted = encryptKey(alicePrivate, primitiveRoot, prime)
print(alicePrivate)
print(aliceEncrypted)

1430723975
43044452


In [ ]:
aliceDecrypted = decryptKey(aliceEncrypted, primitiveRoot, prime)
print(aliceDecrypted)